# TSA Chapter 5: S&P 500 Returns and Volatility

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch5/TSA_ch5_sp500/TSA_ch5_sp500.ipynb)

S&P 500 daily returns with GARCH(1,1) conditional volatility.

In [ ]:
!pip install numpy pandas matplotlib scipy yfinance arch statsmodels -q

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy import stats
# Chart style
BLUE = '#1A3A6E'; RED = '#DC3545'; GREEN = '#2E7D32'
ORANGE = '#E67E22'; GRAY = '#666666'; PURPLE = '#8E44AD'

plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none',
    'savefig.facecolor': 'none', 'savefig.transparent': True,
    'axes.grid': False, 'font.size': 10, 'axes.labelsize': 11,
    'axes.titlesize': 12, 'legend.fontsize': 9, 'xtick.labelsize': 9,
    'ytick.labelsize': 9, 'axes.spines.top': False, 'axes.spines.right': False,
    'lines.linewidth': 1.0, 'axes.linewidth': 0.6,
    'legend.facecolor': 'none', 'legend.framealpha': 0, 'legend.edgecolor': 'none',
    'figure.dpi': 150,
})

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

def legend_bottom(ax, ncol=2, y=-0.18):
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, y), ncol=ncol, frameon=False)

In [ ]:
import yfinance as yf

# Download S&P 500 data
sp = yf.download('^GSPC', start='2000-01-01', end='2025-12-31', progress=False)
if isinstance(sp.columns, pd.MultiIndex):
    sp.columns = sp.columns.get_level_values(0)
sp500_returns = (sp['Close'].pct_change() * 100).dropna()
sp500_returns = pd.Series(sp500_returns.values, index=sp500_returns.index, name='returns')
print(f"S&P 500: {len(sp500_returns)} observations")
from arch import arch_model

# Fit GARCH models on S&P 500
m_g = arch_model(sp500_returns, vol='Garch', p=1, q=1, dist='normal')
r_g = m_g.fit(disp='off')
m_j = arch_model(sp500_returns, vol='Garch', p=1, o=1, q=1, dist='normal')
r_j = m_j.fit(disp='off')
m_e = arch_model(sp500_returns, vol='EGARCH', p=1, q=1, dist='normal')
r_e = m_e.fit(disp='off')
models = {'garch': r_g, 'gjr': r_j, 'egarch': r_e}
print("GARCH models fitted successfully")

In [ ]:
# S&P 500 returns and GARCH volatility
cond_vol = models['garch'].conditional_volatility
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

axes[0].plot(sp500_returns.index, sp500_returns.values, color=BLUE, lw=0.35, alpha=0.8)
axes[0].axhline(0, color='black', lw=0.5)
axes[0].set_ylabel('Daily Return (%)'); axes[0].set_title('S&P 500 - Daily Returns (2000-2025)', fontweight='bold')

events = [('2008-10-15', 'Financial Crisis', 8), ('2020-03-16', 'COVID-19', -12)]
for date, label, y_off in events:
    try:
        idx = pd.Timestamp(date)
        nearest = sp500_returns.index[sp500_returns.index.get_indexer([idx], method='nearest')[0]]
        val = sp500_returns.loc[nearest]
        axes[0].annotate(label, xy=(nearest, val), xytext=(nearest, val + y_off), fontsize=8,
                         color=RED, fontweight='bold', arrowprops=dict(arrowstyle='->', color=RED, lw=0.5))
    except: pass

axes[1].plot(sp500_returns.index, cond_vol, color=RED, lw=0.5)
axes[1].fill_between(sp500_returns.index, 0, cond_vol, color=RED, alpha=0.2)
axes[1].set_ylabel('GARCH Volatility (%)'); axes[1].set_xlabel('Date')
axes[1].set_title('Conditional Volatility GARCH(1,1)', fontweight='bold')
axes[1].axhline(2, color=GREEN, ls='--', lw=0.8, alpha=0.7, label='Normal (~2%)')
axes[1].axhline(4, color=ORANGE, ls='--', lw=0.8, alpha=0.7, label='High (~4%)')

for ax in axes:
    ax.xaxis.set_major_locator(mdates.YearLocator(4))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), ncol=2, frameon=False)
fig.tight_layout(); fig.subplots_adjust(bottom=0.12)
save_chart(fig, 'garch_sp500_returns'); plt.show()